In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

In [2]:

# scrping all the foundation details and links

In [3]:
foundations_url = 'https://www.sephora.com/shop/foundation-makeup'

In [5]:
grids = []

for pagenum in range(1,21):
    page = BeautifulSoup(requests.get((foundations_url+'?currentPage={}&pageSize=12').format(pagenum)).content, 'html.parser')
    grids.append(page.find('div', {'data-comp':'ProductGrid'}))
    

In [6]:
foundation_info = pd.DataFrame(columns=['foundation_name', 'link', 'Brand', 'price', 'colors', 'rank'])

In [7]:
#Get the foundation detils
for grid in grids:
    for row in grid.find_all('div', {'class':"css-dkxsdo"}):
        for product in row.find_all('a', {'class':"css-ix8km1"}):
            if product:
                foundation_name = product.find('span',{'data-at':'sku_item_name'}).text

                rank = product.find('div', {'class':"css-1pjru6n"}).attrs['style']

                try:
                    colors = product.find('div', {'class':'css-rrjz1n'}).text
                except:
                     colors = None
                space_index = product.attrs['href'].index(' ')

                try:
                    price = product.find('span', {'class':'css-0'}).text
                except:
                    price = product.find('span', {'class':'css-1qdampm'}).text

                brand = product.find('span', {'class':'css-ktoumz OneLinkNoTx'}).text

                link = ('https://www.sephora.com' + product.attrs['href'][:space_index])

                foundation_info.loc[len(foundation_info)] = [foundation_name, link, brand, price, colors, rank]
            

AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
len(foundation_info)


In [ ]:
foundation_info.head()

In [ ]:
fondation_info['rank'] = fondation_info['rank'].str.replace('width:', '')
fondation_info['rank'] = fondation_info['rank'].str.replace('%', '')

## Get foundations reviews


In [3]:
# load foundation data
foundation_df = pd.read_csv('Foundation_info.csv')

In [4]:
# show the data
foundation_df.head()

,Unnamed: 0,foundation_name,link,Brand,price,colors,rank
0,0,Pro Filt'r Soft Matte Longwear Foundation,https://www.sephora.com/product/pro-filtr-soft...,FENTY BEAUTY by Rihanna,$34.00,49 more colors,width:81.986%
1,1,Double Wear Stay-in-Place Foundation,https://www.sephora.com/product/double-wear-st...,Estée Lauder,$42.00,55 more colors,width:88.848%
2,2,Ultra HD Invisible Cover Foundation,https://www.sephora.com/product/ultra-hd-invis...,MAKE UP FOR EVER,$19.00 - $43.00,40 more colors,width:78.78999999999999%
3,3,Luminous Silk Foundation,https://www.sephora.com/product/luminous-silk-...,Giorgio Armani Beauty,$42.00 - $64.00,43 more colors,width:85.74199999999999%
4,4,Your Skin But Better™ CC+™ Cream with SPF 50+,https://www.sephora.com/product/your-skin-but-...,IT Cosmetics,$15.00 - $38.00,13 more colors,width:81.32%


In [5]:
# drop the Unnamed column
foundation_df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [6]:
foundation_df.head()

,foundation_name,link,Brand,price,colors,rank
0,Pro Filt'r Soft Matte Longwear Foundation,https://www.sephora.com/product/pro-filtr-soft...,FENTY BEAUTY by Rihanna,$34.00,49 more colors,width:81.986%
1,Double Wear Stay-in-Place Foundation,https://www.sephora.com/product/double-wear-st...,Estée Lauder,$42.00,55 more colors,width:88.848%
2,Ultra HD Invisible Cover Foundation,https://www.sephora.com/product/ultra-hd-invis...,MAKE UP FOR EVER,$19.00 - $43.00,40 more colors,width:78.78999999999999%
3,Luminous Silk Foundation,https://www.sephora.com/product/luminous-silk-...,Giorgio Armani Beauty,$42.00 - $64.00,43 more colors,width:85.74199999999999%
4,Your Skin But Better™ CC+™ Cream with SPF 50+,https://www.sephora.com/product/your-skin-but-...,IT Cosmetics,$15.00 - $38.00,13 more colors,width:81.32%


In [7]:
# Encode each foundation
# EncodedFoundation= {'Abnorml': 1, 'Normal': 0, 'Partial': 0, 'Family' : 0, 'Alloca' : 0, 'AdjLand' : 0}
# train_data['Encoded_Sale_Condition'] = train_data.Sale_Condition.map(EncodedSaleConditionDictionary)

In [8]:
list(foundation_df.link)

['https://www.sephora.com/product/pro-filtr-soft-matte-longwear-foundation-P87985432?icid2=products',
 'https://www.sephora.com/product/double-wear-stay-in-place-makeup-P378284?icid2=products',
 'https://www.sephora.com/product/ultra-hd-invisible-cover-foundation-P398321?icid2=products',
 'https://www.sephora.com/product/luminous-silk-foundation-P393401?icid2=products',
 'https://www.sephora.com/product/your-skin-but-better-cc-cream-spf-50-P411885?icid2=products',
 'https://www.sephora.com/product/born-this-way-P397517?icid2=products',
 'https://www.sephora.com/product/lock-it-tattoo-foundation-P311138?icid2=products',
 'https://www.sephora.com/product/teint-idole-ultra-24h-long-wear-foundation-P308201?icid2=products',
 'https://www.sephora.com/product/bareminerals-original-foundation-broad-spectrum-spf-15-P61003?icid2=products',
 'https://www.sephora.com/product/fauxfilter-foundation-P424302?icid2=products',
 'https://www.sephora.com/product/tinted-moisturizer-broad-spectrum-P109936?i

In [9]:
# import 
from selenium import webdriver

In [21]:
driver = webdriver.Chrome(executable_path='./chromedriver/chromedriver')
driver.get("https://www.sephora.com/product/pro-filtr-soft-matte-longwear-foundation-P87985432?icid2=products")

driver.find_element_by_css_selector('css-zea1jq ')


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"css-zea1jq "}
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.41.578706 (5f725d1b4f0a4acbf5259df887244095596231db),platform=Mac OS X 10.13.6 x86_64)


In [22]:
driver.close()